In [19]:
from model.unifiedGPT import UnifiedGPT,UnifiedGPTConfig
import torch

config = UnifiedGPTConfig(
    block_size=256,
    vocab_size=50304,
    n_layer_main=4,
    n_layer_idea=2,
    n_head=8,
    n_embd_main = 768,          
    n_embd_idea = 512,
    idea_dim=768,
)
model = UnifiedGPT(config)
model.load_state_dict(torch.load("model_checkpoint.pth"))
model.cuda()
model.eval()
model

/tmp/ipykernel_129763/648594184.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_checkpoint.pth"))


UnifiedGPT(
  (wte_i): Embedding(50304, 512)
  (wpe_i): Embedding(256, 512)
  (blocks_i): ModuleList(
    (0-1): 2 x Block(
      (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (c_attn): Linear(in_features=512, out_features=1536, bias=True)
        (c_proj): Linear(in_features=512, out_features=512, bias=True)
      )
      (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Linear(in_features=512, out_features=2048, bias=True)
        (gelu): GELU(approximate='tanh')
        (c_proj): Linear(in_features=2048, out_features=512, bias=True)
      )
    )
  )
  (ln_f_i): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (idea_head): Linear(in_features=512, out_features=768, bias=False)
  (wte_g): Embedding(50304, 768)
  (wpe_g): Embedding(256, 768)
  (blocks_g): ModuleList(
    (0-3): 4 x Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Causal

In [20]:
import tiktoken
# Load the base encoding
enc = tiktoken.get_encoding("gpt2")
eos = 50257
# Define new special tokens
new_special_tokens = {
    "<|endofsent|>": eos,  # Make sure this ID does not conflict with existing tokens
}
# Create a new encoding with the added special tokens
extended_enc = tiktoken.Encoding(
    name="gpt2_extended",
    pat_str=enc._pat_str,  # Use the same pattern as the original encoding
    mergeable_ranks=enc._mergeable_ranks,  # Keep the same mergeable ranks
    special_tokens={**enc._special_tokens, **new_special_tokens},  # Extend special tokens
)

In [21]:
num_return_sequences = 5
max_length = 20
mysent = "A forgotten sock on the kitchen floor can spark a memory of alien landscapes"#, while a half-day dream might hint at interdimensional coffee mugs brimming with cosmic dust"
sent = extended_enc.encode(mysent)
sent = torch.tensor(sent, dtype=torch.long) # (8,)
sent = sent.unsqueeze(0)
sent = sent.cuda()

In [22]:
init_sent = extended_enc.encode("<|endofsent|>", allowed_special={'<|endofsent|>'})
init_sent = torch.tensor(init_sent, dtype=torch.long) # (8,)
init_sent = init_sent.unsqueeze(0).repeat(num_return_sequences, 1)
init_sent = init_sent.cuda()
init_sent

tensor([[50257],
        [50257],
        [50257],
        [50257],
        [50257]], device='cuda:0')

In [23]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

import torch.nn.functional as F

while init_sent.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits, _ = model(init_sent, sent) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        init_sent = torch.cat((init_sent, xcol), dim=1)

In [24]:
for i in range(num_return_sequences):
    tokens = init_sent[i, :max_length].tolist()
    decoded = extended_enc.decode(tokens)
    print(">", decoded)

> <|endofsent|>Sometimes, life feels like a kaleidoscopic blend of distant galaxies and half-eaten sandwiches
> <|endofsent|>Sometimes, life feels like a kaleidoscopic blend of distant galaxies and half-eaten sandwiches
> <|endofsent|>In that confusion, there’s a sort of beauty—each small piece of weirdness
> <|endofsent|>Sometimes, life feels like a kaleidoscopic blend of distant galaxies and half-eaten sandwiches
> <|endofsent|>Sometimes, life feels like a kaleidoscopic blend of distant galaxies and half-eaten sandwiches


In [ ]:
A forgotten sock on the kitchen floor can spark a memory of alien landscapes, while a half-day dream might hint at interdimensional coffee mugs brimming with cosmic dust

In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
new_special_tokens = {
    "<my_special_token>": 100264,
    "<another_token>": 100265,
}

# Add the special tokens
encoding.add_special_tokens(new_special_tokens)

B = 16
T = 1024
dataset = SentenceDataset("data/input.txt", enc)
train_loader = SentenceDataLoaderLite(dataset, block_size=T, B=B)

[nltk_data] Downloading package punkt to /home/longnhat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: SentenceDataLoaderLite.__init__() missing 2 required positional arguments: 'eos' and 'enc'

In [5]:
enc.encode("<|endofsent|>  a")

[27, 91, 437, 1659, 34086, 91, 29, 220, 257]

In [13]:
END_SENT_ID = enc.encode("<|endofsent|>")[0]

tensor([1, 2, 1])

In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

In [ ]:
sd_hf["transformer.wpe.weight"].view(-1)[:20]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(sd_hf["transformer.wpe.weight"], cmap="gray")

In [ ]:
plt.plot(sd_hf["transformer.wpe.weight"][:, 150])
plt.plot(sd_hf["transformer.wpe.weight"][:, 200])
plt.plot(sd_hf["transformer.wpe.weight"][:, 250])

In [ ]:
plt.imshow(sd_hf["transformer.h.1.attn.c_attn.weight"][:300,:300], cmap="gray")

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


In [ ]:
# let's instead sample manually
import torch
from torch.nn import functional as F

model = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
model.eval()
model.to('cuda')
torch.manual_seed(42)
torch.cuda.manual_seed(42)
tokens = [15496, 11, 314, 1101, 257, 3303, 2746, 11] # "Hello, I'm a language model,"
tokens = torch.tensor(tokens, dtype=torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(5, 1) # (5, 8)
x = tokens.to('cuda')

# generate!
while x.size(1) < 30: # max_length=30
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x)[0] # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)

# print the generated text
import tiktoken
enc = tiktoken.get_encoding('gpt2')
for i in range(5):
    tokens = x[i, :30].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

In [ ]:
# tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r') as f:
    text = f.read()
data = text[:1000] # first 1,000 characters
print(data[:100])

In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:24])

In [ ]:
import torch
buf = torch.tensor(tokens[:24 + 1])
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
print(x)
print(y)

In [ ]:
print(sd_hf["lm_head.weight"].shape)
print(sd_hf["transformer.wte.weight"].shape)

In [ ]:
(sd_hf["lm_head.weight"] == sd_hf["transformer.wte.weight"]).all()

In [ ]:
print(sd_hf["lm_head.weight"].data_ptr())
print(sd_hf["transformer.wte.weight"].data_ptr())

In [ ]:

# standard deviation grows inside the residual stream
x = torch.zeros(768)
n = 100 # e.g. 100 layers
for i in range(n):
    x += n**-0.5 * torch.randn(768)

print(x.std())

In [ ]:
import torch

# super simple little MLP
net = torch.nn.Sequential(
    torch.nn.Linear(16, 32),
    torch.nn.GELU(),
    torch.nn.Linear(32, 1)
)
torch.random.manual_seed(42)
x = torch.randn(4, 16)
y = torch.randn(4, 1)
net.zero_grad()
yhat = net(x)
loss = torch.nn.functional.mse_loss(yhat, y)
loss.backward()
print(net[0].weight.grad.view(-1)[:10])

# the loss objective here is (due to readuction='mean')
# L = 1/4 * [
#            (y[0] - yhat[0])**2 +
#            (y[1] - yhat[1])**2 +
#            (y[2] - yhat[2])**2 +
#            (y[3] - yhat[3])**2
#           ]
# NOTE: 1/4!

In [ ]:
# now let's do it with grad_accum_steps of 4, and B=1
# the loss objective here is different because
# accumulation in gradient <---> SUM in loss
# i.e. we instead get:
# L0 = 1/4(y[0] - yhat[0])**2
# L1 = 1/4(y[1] - yhat[1])**2
# L2 = 1/4(y[2] - yhat[2])**2
# L3 = 1/4(y[3] - yhat[3])**2
# L = L0 + L1 + L2 + L3
# NOTE: the "normalizer" of 1/4 is lost
net.zero_grad()
for i in range(4):
    yhat = net(x[i])
    loss = torch.nn.functional.mse_loss(yhat, y[i])
    loss = loss / 4 # <-- have to add back the "normalizer"!
    loss.backward()
print(net[0].weight.grad.view(-1)[:10])


In [ ]:
# parse and visualize the logfile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sz = "124M"

loss_baseline = {
    "124M": 3.2924,
}[sz]
hella2_baseline = { # HellaSwag for GPT-2
    "124M": 0.294463,
    "350M": 0.375224,
    "774M": 0.431986,
    "1558M": 0.488946,
}[sz]
hella3_baseline = { # HellaSwag for GPT-3
    "124M": 0.337,
    "350M": 0.436,
    "774M": 0.510,
    "1558M": 0.547,
}[sz]

# load the log file
with open("log124M_40B/log.txt", "r") as f:
    lines = f.readlines()

# parse the individual lines, group by stream (train,val,hella)
streams = {}
for line in lines:
    step, stream, val = line.strip().split()
    if stream not in streams:
        streams[stream] = {}
    streams[stream][int(step)] = float(val)

# convert each stream from {step: val} to (steps[], vals[])
# so it's easier for plotting
streams_xy = {}
for k, v in streams.items():
    # get all (step, val) items, sort them
    xy = sorted(list(v.items()))
    # unpack the list of tuples to tuple of lists
    streams_xy[k] = list(zip(*xy))

# create figure
plt.figure(figsize=(16, 6))

# Panel 1: losses: both train and val
plt.subplot(121)
xs, ys = streams_xy["train"] # training loss
ys = np.array(ys)
plt.plot(xs, ys, label=f'nanogpt ({sz}) train loss')
print("Min Train Loss:", min(ys))
xs, ys = streams_xy["val"] # validation loss
plt.plot(xs, ys, label=f'nanogpt ({sz}) val loss')
# horizontal line at GPT-2 baseline
if loss_baseline is not None:
    plt.axhline(y=loss_baseline, color='r', linestyle='--', label=f"OpenAI GPT-2 ({sz}) checkpoint val loss")
plt.xlabel("steps")
plt.ylabel("loss")
plt.yscale('log')
plt.ylim(top=4.0)
plt.legend()
plt.title("Loss")
print("Min Validation Loss:", min(ys))

# Panel 2: HellaSwag eval
plt.subplot(122)
xs, ys = streams_xy["hella"] # HellaSwag eval
ys = np.array(ys)
plt.plot(xs, ys, label=f"nanogpt ({sz})")
# horizontal line at GPT-2 baseline
if hella2_baseline:
    plt.axhline(y=hella2_baseline, color='r', linestyle='--', label=f"OpenAI GPT-2 ({sz}) checkpoint")
if hella3_baseline:
    plt.axhline(y=hella3_baseline, color='g', linestyle='--', label=f"OpenAI GPT-3 ({sz}) checkpoint")
plt.xlabel("steps")
plt.ylabel("accuracy")
plt.legend()
plt.title("HellaSwag eval")
print("Max Hellaswag eval:", max(ys))